In [1]:
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109
%load_ext autoreload
%autoreload 2
from AP_class import alch_deriv,first_deriv_nuc_nuc,second_deriv_nuc_nuc
from AP_class import APDFT_perturbator as AP
from AP_symmetry import benz_Symm

In [2]:
benz_atom=""" 
C         6.95191249e-13  2.61173060e+00 -1.67580194e-08 ;
H         2.83052087e-13  4.63941350e+00 -1.15976219e-08 ;
C         2.26182505e+00  1.30586530e+00 -2.37508246e-08 ; 
H         4.01784995e+00  2.31970675e+00 -9.84687205e-09 ; 
C         2.26182505e+00 -1.30586530e+00  8.26065094e-09 ; 
H         4.01784995e+00 -2.31970675e+00  5.83890508e-08 ; 
C        -8.45954456e-13 -2.61173060e+00 -1.55922267e-08 ; 
H         1.49445304e-12 -4.63941350e+00 -3.26774154e-08 ; 
C        -2.26182505e+00 -1.30586530e+00 -8.28266973e-09 ; 
H        -4.01784995e+00 -2.31970675e+00  1.87236306e-09 ; 
C        -2.26182505e+00  1.30586530e+00  1.68546675e-08 ; 
H        -4.01784995e+00  2.31970675e+00  3.31289175e-08 ; 
"""
Benz = gto.M(atom=benz_atom, basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])},unit='bohr')

In [3]:
benz_dm0=np.load("benz_dm.npy")
BenzHF=scf.RHF(Benz)
BenzHF.scf(dm0=benz_dm0)

converged SCF energy = -230.788695455662


-230.7886954556622

In [4]:
apbz=AP(BenzHF,sites=[0,2,4,6,8,10],symm=benz_Symm(Benz))

In [5]:
apbz.build_all()

In [6]:
apbz.APDFT3([1,2,0,0,0,1]),apbz.APDFT3([2,1,1,0,0,0])

(-298.3512228942378, -298.3512228943517)

In [ ]:
apbz.af(0)

No AF found for atom 0. Calculating it from code


In [25]:
apbz.af(2)

array([[-6.22967797e-02,  4.20940361e-02,  1.20191320e-09],
       [-9.74927165e-03,  5.34267417e-03,  7.40513852e-11],
       [ 1.86413894e-02,  1.07626112e-02, -8.55157922e-10],
       [ 6.78848704e-02,  3.91933482e-02,  6.29282072e-11],
       [ 5.30611472e-03, -7.49976118e-02, -9.91093395e-10],
       [-2.47744274e-04, -1.11144540e-02,  3.70406029e-10],
       [ 2.26999491e-03, -2.05165868e-03, -1.87790861e-10],
       [-3.77889933e-03, -4.14951335e-03,  2.06318495e-10],
       [-9.79264947e-03, -5.65378881e-03,  1.81691231e-10],
       [-2.11220030e-03, -1.21947941e-03,  2.29331555e-11],
       [-6.41791082e-04,  2.99170260e-03,  5.69323871e-12],
       [-5.48303364e-03, -1.19786615e-03, -9.18927615e-11]])

In [17]:
from berny import Berny, geomlib, coords,Geometry

In [18]:
apbz.build_all()

In [19]:
pvec_orto=np.asarray([1,-1,0,0,0,0])
pvec_meta=np.asarray([1,0,-1,0,0,0])
pvec_para=np.asarray([1,0,0,-1,0,0])

In [28]:
apbz.APDFT3(pvec_para),apbz.APDFT3(pvec_orto),apbz.APDFT3(pvec_meta)

(-234.18924250266744, -234.3009028753131, -234.2759951827738)

In [31]:
charge2symbol={1:"H",2:"He",3:"Li",4:"Be",5:"B",6:"C",7:"N",8:"O",9:"F",10:"Ne"}

In [32]:
mp_orto=apbz.targ_mol(np.asarray([apbz.sites,pvec_orto]))
pbg_orto=Geometry([charge2symbol[x] for x in mp_orto.atom_charges()],mp_orto.atom_coords()*bohr2ang)
b_orto=Berny(pbg_orto,debug=True)
b_orto.send((apbz.APDFT3(pvec_orto),apbz.af(0)-apbz.af(2)))  #  fcs=[0,2],[1,-1]

{'geom': <Geometry 'BC4H6N'>,
 'params': {'gradientmax': 0.00045,
  'gradientrms': 0.00015,
  'stepmax': 0.0018,
  'steprms': 0.0012,
  'trust': 0.3,
  'dihedral': True,
  'superweakdih': False},
 'trust': 0.3,
 'coords': <InternalCoords "bonds: 12, angles: 18, dihedrals: 24">,
 'H': array([[0.47327986, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.50722226, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.49274515, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.00656333, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.00648628,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.00633378]]),
 'weights': array([1.05173303, 1.12716058, 1.09498923, 1.11163327, 1.13971091,
        1.06924566, 1.10799607, 1.06924566, 1.

In [45]:
b_orto._state.geom.dump(open("1_pred_orto.xyz","w"),"xyz")

In [54]:
pbg_orto.dump(open("1_orto_benzgeom.xyz","w"),"xyz")

In [47]:
Benz.tofile("benz_opt.xyz")

'12\nXYZ from PySCF\nC           0.00000        1.38207       -0.00000\nH           0.00000        2.45507       -0.00000\nC           1.19691        0.69103       -0.00000\nH           2.12615        1.22754       -0.00000\nC           1.19691       -0.69103        0.00000\nH           2.12615       -1.22754        0.00000\nC          -0.00000       -1.38207       -0.00000\nH           0.00000       -2.45507       -0.00000\nC          -1.19691       -0.69103       -0.00000\nH          -2.12615       -1.22754        0.00000\nC          -1.19691        0.69103        0.00000\nH          -2.12615        1.22754        0.00000'

In [48]:
import rmsd 

In [52]:
rmsd.main(args=["./1_pred_orto.xyz","../Optimizing_structures/1_orto.xyz"])

0.04802596960271668


In [55]:
rmsd.main(args=["./1_orto_benzgeom.xyz","../Optimizing_structures/1_orto.xyz"])

0.0997748966980053


In [58]:
mp_meta=apbz.targ_mol(np.asarray([apbz.sites,pvec_meta]))
pbg_meta=Geometry([charge2symbol[x] for x in mp_meta.atom_charges()],mp_meta.atom_coords()*bohr2ang)
b_meta=Berny(pbg_meta)
b_meta.send((apbz.APDFT3(pvec_meta),apbz.af(0)-apbz.af(4)))  #  fcs=[0,2],[1,-1]
b_meta._state.geom.dump(open("2_pred_meta.xyz","w"),"xyz")
pbg_meta.dump(open("2_meta_benzgeom.xyz","w"),"xyz")

In [59]:
rmsd.main(args=["./2_pred_meta.xyz","../Optimizing_structures/2_meta.xyz"])

0.06469408456292522


In [60]:
rmsd.main(args=["./2_meta_benzgeom.xyz","../Optimizing_structures/2_meta.xyz"])

0.13267514530940733


In [61]:
mp_para=apbz.targ_mol(np.asarray([apbz.sites,pvec_para]))
pbg_para=Geometry([charge2symbol[x] for x in mp_para.atom_charges()],mp_para.atom_coords()*bohr2ang)
b_para=Berny(pbg_para)
b_para.send((apbz.APDFT3(pvec_para),apbz.af(0)-apbz.af(6)))  #  fcs=[0,2],[1,-1]
b_para._state.geom.dump(open("3_pred_para.xyz","w"),"xyz")
pbg_para.dump(open("3_para_benzgeom.xyz","w"),"xyz")

In [62]:
rmsd.main(args=["./3_pred_para.xyz","../Optimizing_structures/3_para.xyz"])

0.0689631285185073


In [64]:
rmsd.main(args=["./3_para_benzgeom.xyz","../Optimizing_structures/3_para.xyz"])

0.13792607095981138
